<a href="https://colab.research.google.com/github/NagaHimaDeepthiGollamudi/Internship-coincent/blob/main/internship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
import numpy as np

In [ ]:
with open("train_qa.txt","rb") as fp:
  train_data=pickle.load(fp)

In [ ]:
train_data

In [ ]:
with open("test_qa.txt","rb") as fp:
  test_data=pickle.load(fp)

In [ ]:
test_data

In [ ]:
type(train_data)

list

In [ ]:
type(test_data)

list

In [ ]:
len(train_data)

1000

In [ ]:
len(test_data)

1000

In [ ]:
train_data[0]

In [ ]:
' '.join(train_data[0][0])

'Mary got the milk there . John moved to the bedroom .'

In [ ]:
' '.join(train_data[0][1])

'Is John in the kitchen ?'

In [ ]:
''.join(train_data[0][2])

'no'

In [ ]:
vocab=set()

In [ ]:
total_data=train_data+test_data

In [ ]:
total_data

In [ ]:
type(total_data)

list

In [ ]:
len(total_data)

2000

In [ ]:
for story, question, answer in total_data:
  vocab=vocab.union(set(story))
  vocab=vocab.union(set(question))

In [ ]:
vocab.add("yes")
vocab.add("no")

In [ ]:
vocab

In [ ]:
vocab_len=len(vocab)+1
vocab_len

38

In [ ]:
for data in total_data:
  print(len(data[0]))

In [ ]:
max_story_data=max(len(data[0]) for data in total_data)

In [ ]:
max_story_data

86

In [ ]:
max_question_data=max(len(data[1]) for data in total_data)
max_question_data

6

In [ ]:
max_answer_data=max(len(data[2]) for data in total_data)
max_answer_data

3

In [ ]:

from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)
tokenizer.word_index

In [ ]:
train_story_text=[]
train_question_text=[]
train_answers=[]

for story,question,answer in train_data:
  train_story_text.append(story)
  train_question_text.append(question)

In [ ]:
train_story_seq=tokenizer.texts_to_sequences(train_story_text)
len(train_story_text)

1000

In [ ]:
len(train_story_seq)

1000

In [ ]:
train_story_seq

In [ ]:
train_story_text

In [ ]:
def vectorize_stories(data,word_index=tokenizer.word_index,
                      max_story_len=max_story_data,max_ques_len=max_question_data):
  X=[]#story
  Xq=[]#question quary
  Y=[]#correct answer

  for story,quary,answer in data:
    x= [word_index[word.lower()] for word in story]
    xq= [word_index[word.lower()] for word in quary]
    y=np.zeros(len(word_index)+1)
    y[word_index[answer]]=1

    X.append(x)
    Xq.append(xq)
    Y.append(y)

  return(pad_sequences(X,maxlen=max_story_len),
         pad_sequences(X,maxlen=max_ques_len),
         np.array(Y))

In [ ]:
inputs_train,queries_train,answers_train = vectorize_stories(train_data)

In [ ]:
inputs_test,queries_test,answers_test = vectorize_stories(test_data)

In [ ]:
inputs_train

In [ ]:
queries_test

In [ ]:
tokenizer.word_index['yes']

32

In [ ]:
tokenizer.word_index['no']

21

In [ ]:
from keras.models import Sequential,Model
from keras.layers import Embedding
from keras.layers import Input,Activation,Dense,Permute,Dropout,add,dot,concatenate,LSTM

In [ ]:
input_sequence=Input((max_story_data,))
question=Input((max_question_data,))

In [ ]:
input_encoder_m=Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_len,output_dim=64))
input_encoder_m.add(Dropout(0.3))

In [ ]:
input_encoder_c=Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_len,output_dim=max_question_data))
input_encoder_c.add(Dropout(0.3))


In [ ]:
#question encoder
question_encoder=Sequential()
question_encoder.add(Embedding(input_dim=vocab_len,output_dim=64,input_length=max_question_data))
question_encoder.add(Dropout(0.3))

In [ ]:
input_encoded_m=input_encoder_m(input_sequence)

input_encoded_c=input_encoder_c(input_sequence)
question_encoded=question_encoder(question)

In [ ]:
match=dot([input_encoded_m,question_encoded],axes=(2,2))
match= Activation ('softmax')(match)

In [ ]:
response=add([match,input_encoded_c])
response=Permute((2,1))(response)

In [ ]:
answer=concatenate([response,question_encoded])

In [ ]:
answer

<KerasTensor: shape=(None, 6, 150) dtype=float32 (created by layer 'concatenate_2')>

In [ ]:
answer=LSTM(32)(answer)

In [ ]:
answer=Dropout(0.5)(answer)
answer=Dense(vocab_len)(answer)

In [ ]:
answer=Activation('softmax')(answer)

In [ ]:
model=Model([input_sequence,question],answer)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history=model.fit([inputs_train,queries_train],answers_train,
                  batch_size=32,epochs=20,validation_data=([inputs_test,queries_test],answers_test))

In [ ]:
import matplotlib.pyplot as plt
print(history.history.keys())
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model Accuracy")
plt.xlabel("epochs")

In [ ]:
model.save("chatbot_model")

In [ ]:
model.load_weights("chatbot_model")

In [ ]:
pred_results=model.predict(([inputs_test,queries_test]))

32/32 [==============================] - 1s 5ms/step


In [ ]:
test_data[0][0]

In [ ]:
story=' '.join(word for word in test_data[0][0])
story

'Mary got the milk there . John moved to the bedroom .'

In [ ]:
query=' '.join(word for word in test_data[0][1])
query

'Is John in the kitchen ?'

In [ ]:
test_data[13][2]

'yes'

In [ ]:
story=' '.join(word for word in test_data[0][0])
story

'Mary got the milk there . John moved to the bedroom .'

In [ ]:
query=' '.join(word for word in test_data[10][1])
query

'Is John in the hallway ?'

In [ ]:
val_max=np.argmax(pred_results[0])

for key,val in tokenizer.word_index.items():
  if val==val_max:
    k=key

print("predicted Answer is",k)
print("probability of certainity",pred_results[0][val_max])

predicted Answer is no
probability of certainity 0.51972866


In [ ]:
story="Mary dropped football . Sandra discarded apple in kitchen"
story.split()

['Mary',
 'dropped',
 'football',
 '.',
 'Sandra',
 'discarded',
 'apple',
 'in',
 'kitchen']

In [ ]:
my_question="Is apple in the kitchen ?"
my_question.split()

['Is', 'apple', 'in', 'the', 'kitchen', '?']

In [ ]:
mydata=[(story.split(),my_question.split(),'yes')]
my_story,my_ques,my_ans=vectorize_stories(mydata)
pred_results=model.predict(([my_story,my_ques]))

val_max=np.argmax(pred_results[0])

for key,val in tokenizer.word_index.items():
  if val==val_max:
    k=key

print("predicted Answer is",k)
print("probability of certainity",pred_results[0][val_max])

1/1 [==============================] - 0s 26ms/step
predicted Answer is yes
probability of certainity 0.5142411
